# Depuracion de datos

In [1]:
import pandas as pd
import os
import pickle
from sklearn.preprocessing import FunctionTransformer
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_csv("datos_limpios.csv")

In [3]:
df

,id_extraccion,timestamp_extraccion,marca,es_demo,es_km0,año_matriculacion,combustible,kilometraje,ubicacion,precio_contado,...,latitud,longitud,garantia,transmision,potencia_cv,puertas,asientos,tipo_carroceria,mes_matriculacion,modelo
0,https://www.autocasion.com/coches-segunda-mano...,2025-06-20 21:10:20.086439,ABARTH,0,0,2022,Gasolina,23213.0,Madrid,22990.0,...,40.4168,-3.7038,12,Manual,179.0,2.0,NaN,Descapotable,12.0,500 C 695C 1.4 16v T-Jet 132kW (180 CV)
1,https://www.autocasion.com/coches-segunda-mano...,2025-06-20 21:10:20.086694,ABARTH,0,0,2018,Gasolina,97814.0,Barcelona,23990.0,...,41.3851,2.1734,12,Automático,170.0,2.0,2.0,Descapotable,12.0,124 SPIDER Spider Turbo Multiair 125kW Auto
2,https://www.autocasion.com/coches-segunda-mano...,2025-06-20 21:10:20.086996,ABARTH,0,0,2022,Gasolina,33602.0,Malaga,22500.0,...,36.7213,-4.4214,12,Manual,165.0,3.0,4.0,Berlina,3.0,595 1.4T JET 121KW
3,https://www.autocasion.com/coches-segunda-mano...,2025-06-20 21:10:20.087274,ABARTH,0,0,2018,Gasolina,48500.0,Islas Baleares,22950.0,...,39.5696,2.6502,12,Manual,180.0,3.0,4.0,Pequeño,2.0,695 Pequeño Manual de 3 Puertas
4,https://www.autocasion.com/coches-segunda-mano...,2025-06-20 21:10:20.087595,ABARTH,0,0,2020,Gasolina,32922.0,Barcelona,16999.0,...,41.3851,2.1734,12,Manual,145.0,3.0,NaN,Berlina,7.0,500 1.4 Turbo 595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88419,https://www.autocasion.com/coches-km0/km-0/yud...,2025-06-21 00:33:12.156646,YOOUDOOO,0,0,2025,Gasolina,2.0,La Rioja,17590.0,...,42.4627,-2.4450,36,Manual,109.0,5.0,5.0,Todo Terreno,2.0,Y2 1.5L Comfort
88420,https://www.autocasion.com/coches-km0/km-0/yud...,2025-06-21 00:33:12.156777,YOOUDOOO,0,0,2024,Eléctrico,10.0,Barcelona,26838.0,...,41.3851,2.1734,12,Automático,95.0,5.0,5.0,Pequeño,8.0,K3 YUDO Pequeño Automático de 5 Puertas
88421,https://www.autocasion.com/coches-km0/km-0/yud...,2025-06-21 00:33:12.156928,YOOUDOOO,0,0,2024,Eléctrico,3000.0,Sevilla,21900.0,...,37.3891,-5.9845,96,Automático,95.0,5.0,5.0,Todo Terreno,7.0,K3 70kW
88422,https://www.autocasion.com/coches-km0/km-0/yud...,2025-06-21 00:33:12.157050,YOOUDOOO,0,0,2025,Eléctrico,4585.0,La Rioja,22990.0,...,42.4627,-2.4450,84,Automático,95.0,5.0,5.0,Todo Terreno,1.0,K3 70kW


* Imputamos NaNs con la moda de las columnas categoricas

In [4]:
categorical_cols = ['combustible', 'transmision', 'tipo_carroceria']

imputers = {}

for col in categorical_cols:
    imputer = SimpleImputer(strategy='most_frequent')
    df[[col]] = imputer.fit_transform(df[[col]])
    imputers[col] = imputer
    
    with open(f'encoders/imputer_{col}.pkl', 'wb') as f:
        pickle.dump(imputer, f)


* Imputamos NaNs de columnas numericas con KNNImputer

In [5]:
num_cols_to_impute = ['mes_matriculacion', 'potencia_cv', 'puertas', 'asientos']

knn_imputer = KNNImputer(n_neighbors=3)

df[num_cols_to_impute] = knn_imputer.fit_transform(df[num_cols_to_impute])

with open('encoders/knn_imputer_num_cols.pkl', 'wb') as f:
    pickle.dump(knn_imputer, f)


* Cambiamos el tipo de dato a las columnas seleccionadas despues de imputar NaNs ya que con el KNNImputer te deja los numeros en decimales 

In [6]:
cols_int = ['mes_matriculacion', 'potencia_cv', 'puertas', 'asientos']

for col in cols_int:
    df[col] = df[col].round().astype('Int64')


* Hacemos target encoding de las columnas "marca" y "modelo"

In [7]:
def target_encode(train_df, col, target):
    encoding_map = train_df.groupby(col)[target].mean()
    
    train_df[col + '_te'] = train_df[col].map(encoding_map)
    
    with open(f'encoders/target_encoding_{col}.pkl', 'wb') as f:
        pickle.dump(encoding_map, f)
    
    return train_df

df = target_encode(df, 'marca', 'precio_contado')
df = target_encode(df, 'modelo', 'precio_contado')

df = df.drop(['marca', 'modelo'], axis=1)


* Hacemos encoding binario de la columna "transmision" ya que solo contiene dos valores diferentes

In [8]:
transmision_map = {'Manual': 0, 'Automático': 1}

df['transmision_bin'] = df['transmision'].map(transmision_map)

df.drop(columns=['transmision'], inplace=True)

with open('encoders/bin_encoder_transmision.pkl', 'wb') as f:
    pickle.dump(transmision_map, f)


* Hacemos One Hot Encoding para la columna "combustible"

In [9]:

ohe_combustible = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
combustible_encoded = ohe_combustible.fit_transform(df[['combustible']])

combustible_cols = [f"combustible_{cat}" for cat in ohe_combustible.categories_[0]]
df_combustible = pd.DataFrame(combustible_encoded, columns=combustible_cols, index=df.index)

df = pd.concat([df, df_combustible], axis=1)
df.drop(columns=['combustible'], inplace=True)

with open('encoders/ohe_combustible.pkl', 'wb') as f:
    pickle.dump(ohe_combustible, f)


* One Hot Encoding de la columna "tipo_carroceria"

In [10]:
ohe_carroceria = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
carroceria_encoded = ohe_carroceria.fit_transform(df[['tipo_carroceria']])

carroceria_cols = [f"tipo_carroceria_{cat}" for cat in ohe_carroceria.categories_[0]]
df_carroceria = pd.DataFrame(carroceria_encoded, columns=carroceria_cols, index=df.index)

df = pd.concat([df, df_carroceria], axis=1)
df.drop(columns=['tipo_carroceria'], inplace=True)

with open('encoders/ohe_tipo_carroceria.pkl', 'wb') as f:
    pickle.dump(ohe_carroceria, f)

* Eliminamos columnas que no necesitamos para el modelo 

In [11]:
df.drop(columns=[
    'id_extraccion',
    'timestamp_extraccion',
    'ubicacion'
], inplace=True)


* Separamos la variable objetivo ("y") del resto de las variables predictoras ("X"). En "X" se almacenan todas las columnas excepto el target, y en "y" únicamente la columna "precio_contado".
* Aplicamos "StandardScaler" a "X" para escalar todas las variables predictoras, lo que garantiza que tengan media 0 y desviación estándar 1, mejorando el rendimiento de muchos modelos.


In [12]:
X = df.drop(columns=['precio_contado'])
y = df['precio_contado']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

with open("encoders/standard_scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns, index=df.index)


* Creamos un nuevo DataFrame ("df_final") que combina las variables escaladas con la variable objetivo ("precio_contado") y lo guardamos en un .csv

In [13]:
df_final = X_scaled_df.copy()
df_final['precio_contado'] = y
df_final.to_csv('datos_limpios_modelo.csv', index=False)


* Dividimos los datos en conjuntos de entrenamiento y prueba usando "train_test_split"

In [14]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled_df, y, test_size=0.2, random_state=42
)

print(f"X_train: {X_train.shape}, X_test: {X_test.shape}")
print(f"y_train: {y_train.shape}, y_test: {y_test.shape}")


X_train: (70739, 30), X_test: (17685, 30)
y_train: (70739,), y_test: (17685,)
